# Deploy a un endpoint local

In [ ]:
! pip show azure-ai-ml

## Conectar a workspace

In [ ]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

## Definir y crear un endpoint

Definir el endpoint. 
> Tener en cuenta que el nombre debe ser único: Usaremos la función `dateTime` para ello.

In [5]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Local endpoint for MLflow diabetes model",
    auth_mode="key",
)

Crear el endpoint:

In [6]:
ml_client.begin_create_or_update(endpoint, local=True).result()

DockerEngineNotAvailableError: Please make sure Docker Engine is installed and running. https://docs.docker.com/engine/install/

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> ¡IMPORTANTE!¡Esperar hasta que el endpoint se cree con éxito antes de continuar!
Una notificación verde debería aparecer en el estudio. </p>



## Configurar el deployment gestionado

In [ ]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

# create a blue deployment
model = Model(
    path="./model",
    type=AssetTypes.MLFLOW_MODEL,
    description="my sample mlflow model",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="STANDARD_E2S_V3",
    instance_count=1,
)

## Crear el deployment

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Asignar el 100% del tráfico.

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

## Probar el deploymnent

In [ ]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

## Listar los endpoints

In [ ]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

## Obtener detalles de un endpoint

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## Borrar el endpoint y deployment

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)